In [32]:
import pandas as pd
import codecs
from hazm import *
from case_retrieval import BM25Retrival


In [27]:
import numpy as np
similarity = np.load("../../resource/case/case_similarity.npy")

In [82]:
import pandas as pd
case_df = pd.read_csv("../../resource/case/case.csv", encoding="utf-8")
ri_model = BM25Retrival(case_df.text[1000:1500], [], word_tokenize)

In [101]:
from nltk.translate.bleu_score import corpus_bleu
import itertools, warnings
def bleu_similarity(docs):
    warnings.filterwarnings("ignore")
    score= np.zeros((len(docs),len(docs)))
    for i in tqdm(range(len(docs))):
        for j in range(len(docs)):
            candidate = word_tokenize(docs[i])
            reference = [word_tokenize(docs[j])]
            score[i,j] = corpus_bleu([reference],[candidate],weights=[1,1,0,0])
    return score

In [102]:
belu_score = bleu_similarity(case_df.text[1000:1500].tolist())

100%|██████████| 500/500 [35:14<00:00,  4.23s/it]


In [110]:
from tqdm import tqdm
sorted_indices = np.argsort(-similarity)
sorted_similarity=sorted_indices[:,1:11]
bleu_indices = np.argsort(-belu_score)
bleu_similarity=bleu_indices[:,1:11]
accept_item = []
for i in tqdm(range(500)):
    # llm_result = sorted_similarity[i,:].tolist()
    bm25_result = bleu_similarity[i,:].tolist()
    bm25_sim = ri_model.get_similar(case_df.text[i+1000])[1:11]
    llm_result = [item['corpus_id'] for item in bm25_sim ]
    if any(item in bm25_result for item in llm_result):
        accept_item.append(i)

100%|██████████| 500/500 [02:42<00:00,  3.08it/s]


In [111]:
print(len(accept_item))

475


In [60]:
list1 = [1, 2, 3, 4, 5]
list2 = [4, 5, 6, 7, 8]

result = any(item in list2 for item in list1)
print(result)  # Output will be True


True


In [52]:
ri_model.get_similar(case_df.text[1000])[1:6][0]["corpus_id"]

3

In [14]:
case_df.to_csv("../../resource/case/case.csv", encoding="utf-8", index=False)

In [21]:
case_list = case_df.text.tolist()[1000:1500]
case_text = '<#>'.join(case_list)


In [23]:
with open("../../resource/case/case.txt", 'w', encoding="utf-8") as file:
    file.write(case_text)

In [24]:
case_list = case_text.split("<#>")

In [25]:
len(case_list)

500

In [81]:
h_normalizer = Normalizer()
stopwords = [h_normalizer.normalize(x.strip()) 
             for x in codecs.open("../../config/case_stopwords.txt",'r','utf-8').readlines()]

In [14]:
case_df = pd.read_csv("../../resource/case/case.csv")
nonstop_case = []
for text_tok in case_df.text :   
    nonstop_case.append(' '.join([tok for tok in word_tokenize(text_tok) if tok not in stopwords]))


In [15]:
nonstop_case_df = pd.DataFrame({"nonstop_text": nonstop_case})
nonstop_case_df.to_csv("../../resource/case/nonstop_case.csv")

In [16]:
nonstop_case_df.loc[0,"nonstop_text"]

'رأی شعبه بدوی دیوان عدالت اداری شکایت خانم م م ب طرفیت سازمان تأمین اجتماعی خواسته فوق\u200cالذکر [ لزام اصلاح میزان سختی کار ونوبت کاری افزایش ضریب ریالی دولت ۸۷ ۱ ۱ وپرداخت کلیه معوقات بشرح متن دادخواست ] وفق مواد ۹ ۱۴ اساسنامه سازمان تأمین اجتماعی بررسی اتخاذ تصمیم برنامه بودجه صورتهای مالی سازمان شورا هیأت مدیره عنایت دفاعیات شکایت فوق\u200cالعاده\u200cهای خواسته الزاما مبتنی امتیازدهی عملکرد بدوا امتیازدهی تصویب مراجع ذیصلاح قرار زمانیکه ضوابط اجرایی مصوب طریق مراجع ذیصلاح قانونی تصویب ابلاغ قرار نگیرد محاسبه پرداخت میسور نمی\u200cباشد شکایت نحو مطروحه قابلیت استماع نداشته مستندا ماده ۵۳ قانون تشکیلات آئین دادرسی دیوان عدالت اداری رأی رد صادر اعلام می\u200cنماید رأی ظرف ۲۰ روز تاریخ ابلاغ تجدید شعب تجدید دیوان عدالت اداری دادرس شعبه ۲۱ دیوان عدالت اداری بهرامی رأی شعبه تجدیدنظر دیوان عدالت اداری مداقه اوراق محتوای پرونده این\u200cکه دادنامه صادره شعبه بدوی دیوان رعایت تشریفات قانونی اصدار دلیل قانونی نقض نداشته ناحیه تجدید نظرخواه ایرادی مخدوش رأی معترض عنه ابراز شعبه نهم تجدید د

In [34]:
case_df = pd.read_csv("../../resource/case/case.csv")
titles = case_df.title.tolist()
textes = case_df.text.tolist()
title_score = 3
docs = [] 
def make_scored_text(title, text):
    return (title+" ")*title_score+text
for i in range(len(titles)):
    docs.append(make_scored_text(titles[i], textes[i]))


In [35]:
ri_model = BM25Retrival(docs, stopwords, word_tokenize)


In [36]:
output = ri_model.get_similar(make_scored_text(titles[1001], textes[1001]))

In [37]:
print(output[:10])

[{'corpus_id': 1001, 'score': 2030.7953045182965}, {'corpus_id': 911, 'score': 805.9209061733029}, {'corpus_id': 1003, 'score': 746.9617677581366}, {'corpus_id': 1011, 'score': 701.0897668789672}, {'corpus_id': 692, 'score': 679.9349744644379}, {'corpus_id': 1047, 'score': 679.3680199407789}, {'corpus_id': 1064, 'score': 666.5626519559177}, {'corpus_id': 1000, 'score': 665.7411360789133}, {'corpus_id': 449, 'score': 658.3831272863246}, {'corpus_id': 842, 'score': 656.6665378366768}]


In [30]:
print(case_df.loc[911,"text"])


. رأی دادگاه بدوی. در خصوص دعوی آقای م. الف. فرزند م. با وکالت آقای ش. ق. بطرفیت ۱ -شرکت توزیع نیروی برق تهران بزرگ ۲ - شرکت توزیع نیروی برق شمال شرق تهران ۳ - شرکت مادر تخصصی تولید و انتقال و توزیع نیروی برق ایران (توانیر) به خواسته الزام خواندگان به پرداخت قیمت کارشناسی روز یک قطعه زمین از پلاک ثبتی … واقع در بخش ۱۱ تهران به انضمام خسارات دادرسی، بدین شرح که خواهان مالک شش‌دانگ یک قطعه زمین به مساحت ۱۶۰۰ متر مربع به پلاک ثبتی فوق‌الذکر واقع در تهران … می‌باشد که حدود ۴۸ متر مربع از ملک فوق توسط خواندگان تصرف و در آن احداث پست برق صورت گرفته است ولی خواندگان علیرغم درخواست خواهان نسبت به پرداخت بهای ملک مورد تصرف اقدام ننموده‌اند لذا خواهان تقاضای صدور حکم به شرح خواسته را نموده است. همچنین دادخواست دیگری تحت عنوان تقابل توسط شرکت توزیع نیروی برق تهران بزرگ بامدیریت عاملی آقای م. ه. بطرفیت آقای م. الف. فرزند م. به خواسته دعوی تقابل و الزام خوانده به تفکیک و تنظیم سند رسمی انتقال محل پست برق مساحت ۴۸ متر مربع از پلاک ثبتی … فرعی از … اصلی بخش ۱۱ تهران به انضمام خسارات دادرسی؛ بدین شرح

# next section

In [2]:
case_df = pd.read_csv("../../resource/case/case.csv")
ri_model = SyntacticalRetrival(case_df.head(100))

NameError: name 'SyntacticalRetrival' is not defined

In [12]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from hazm import *

from operator import itemgetter
 

import codecs
stopwords = [h_normalizer.normalize(x.strip()) 
             for x in codecs.open("../../config/case_stopwords.txt",'r','utf-8').readlines()]

# Sample documents


# Preprocessing - you may want to tokenize, remove stopwords, etc.

# Train Word2Vec model
documents = case_df.title
tokenized_docs = [word_tokenize(doc) for doc in documents]
tokenized_docs_nonstopwords = [tok for tok in tokenized_docs if tok not in stopwords]
model = Word2Vec(tokenized_docs_nonstopwords, vector_size=128, window=10, min_count=1, workers=4)

# Function to get document embeddings
def get_doc_embedding(doc, model):
    word_embeddings = [model.wv[word] for word in doc if word in model.wv]
    if len(word_embeddings) > 0:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)  # Return zero vector if no words found in vocab

# Get embeddings for all documents
doc_embeddings = [get_doc_embedding(word_tokenize(doc), model) for doc in documents]

# Function to calculate cosine similarity between two vectors
def calculate_cosine_similarity(vector1, vector2):
    return cosine_similarity([vector1], [vector2])[0][0]

def get_similarty(query:str):
    similarities = [calculate_cosine_similarity(get_doc_embedding(word_tokenize(query), model), doc_emb) for doc_emb in doc_embeddings]
    most_similar_index = np.argmax(similarities) 
    indices = list(sorted(enumerate(similarities), key = itemgetter(1)))[-5:]
    return indices




In [13]:
# similarities = get_similarty(case_df.loc[101,"text"])
# similarities.index(max(similarities))
get_similarty(case_df.loc[1001,"title"])

# print(model.wv.most_similar('قانون', topn=10) )


[(859, 0.99995995),
 (594, 0.9999603),
 (352, 0.99996054),
 (617, 0.99996054),
 (1001, 1.0)]

In [21]:
vector1 = model.wv["ارای"]
vector2 = model.wv["وحدت"]
print(cosine_similarity([vector1], [vector2])[0][0])

0.146308


In [15]:
print(case_df.loc[155,"text"])
# case_df.loc[199,"number"])


. رأی شعبه بدوی دیوان عدالت اداری. در خصوص شکایت مطروحه به شرح فوق‌الذکر [به طرفیت کمیسیون ماده ۲۰ معاونت غذا و داروی دانشگاه علوم پزشکی شهید بهشتی و موضوع شکایت و خواسته ابطال رای مورخه ۹ / ۱۱ / ۹۱ کمیسیون ماده ۲۰ و الزام خوانده به صدور پروانه تاسیس سرمایه گذاری داروخانه] با توجه به مدارک ابرازی و لایحه دفاعیه طرف شکایت اولا شاکی درخواست انتقال پروانه متعلق به پدرش را به نام خود دارد که تاسیس جدید محسوب نمی‌گردد. ثانیا مطابق آراء شماره ۵ مورخ ۶۲ / ۳ / ۹ و ۹۷ - ۹۶ مورخ ۸۰ / ۳ / ۴ و ۲۴۰ - ۸۴ / ۵ / ۳۰ صادره از هیئت عمومی دیوان عدالت اداری موسس نیازی به داشتن مدرک تخصصی و فنی خاص نداشته و فروش و انتقال پروانه به غیر داروساز نیز منعی ندارد. ثالثا تخلفات اعلامی توسط طرف شکایت بر فرض صحت مربوط به مسئولین فنی داروخانه و نسخه پیچ می‌باشد. رابعا مراجع مربوطه مکلف به اجرای آراء صادره از شعب و هیئت عمومی دیوان عدالت اداری می‌باشند که در مانحن فیه در دادنامه شماره ۲۵۱۷ - ۹۰ / ۱۰ / ۷ صادره از این شعبه در سطر آخر صراحتا به صدور پروانه تاسیس و اجابت خواسته شاکی اظهارنظر گردیده است ولی اعضاء کمیسیون 